In [1]:
import os
import json
from pprint import pprint, pformat
from experiments.audio_control import *
from experiments.motor_control import *

CLAUDE_API_URL = os.getenv("CLAUDE_API_URL")
CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")
CLAUDE_API_VER = os.getenv("CLAUDE_API_VER")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# byte_stream = get_audio_instruction()
# task = transcribe(byte_stream)
task = "Please turn left and then turn right twice."
print(task)

dummy_motor, dummy_funcs = get_motor_funcs(dummy=True)
# real_motor, real_funcs = get_motor_funcs(dummy=False)

thought_actions, answer, messages = get_action_steps(task, funcs=dummy_funcs)
pprint(thought_actions)

if False:
    for thought_action in thought_actions:
        step = thought_action.get("Action")
        obs = invoke_tool(step, real_funcs)
    
    if answer:
        print(f"Final answer: {answer}")

Please turn left and then turn right twice.
{'blocking': True, 'degrees': 90, 'name': 'run_for_degrees', 'speed': -50}
{'blocking': True, 'degrees': 90, 'name': 'run_for_degrees', 'speed': 50}
{'blocking': True, 'degrees': 90, 'name': 'run_for_degrees', 'speed': 50}
[{'Action': {'blocking': True,
             'degrees': 90,
             'name': 'run_for_degrees',
             'speed': -50},
  'Thought': 'I need to break this down into three steps: 1) turn left (90 '
             'degrees), 2) turn right (90 degrees), and 3) turn right again '
             "(90 degrees). Since turning involves rotation, I'll use "
             "run_for_degrees. For left turn, I'll use negative speed, and for "
             'right turns, positive speed.'},
 {'Action': {'blocking': True,
             'degrees': 90,
             'name': 'run_for_degrees',
             'speed': 50},
  'Thought': 'Good, the left turn is complete. Now I need to execute the first '
             'right turn.'},
 {'Action': {'bl

In [5]:
# task = "Please turn left."

def control(real_funcs, dummy_funcs):
    """ Runs an audio input loop

    Args:
        :param real_funcs: dict
            References to motor functions to actually perform the physical actions
        :param dummy_funcs: dict
            References to dummy motor functions that will be used to only generate execution plans
    
    Loops over the following steps:
        1. Prints 'Press [return] to speak, or 'q' to exit: ' and waits for user input
        2. If user inputs `q` in step 1, then program quits
        3. If user presses `[return]` key in setp 1, then program starts recording 4 secs audio
        4. Recorded audio is transcribed to text and the text is set as the task
        5. The transcribed text is converted back to audio and output through speaker; user is asked to confirm the task
        6. If user confirms, a plan is generated for the task (transcribed text)
        7. The plan is executed
        8. Final result of the task is announced through audio speaker
        9. Go to Step 1
    """
    answer = None
    for i in range(5):
        inp = input("Press [return] to speak, or 'q' to exit: ")
        
        if inp.lower().startswith("q"):
            print("'q' pressed... Exiting")
            break
        
        byte_stream = get_audio_instruction(duration=4)
        task = transcribe(byte_stream)
        print(f"\nTask: {task}\n")
        if True:
            audio = speak(f"You said: {task}. Is that correct?")
            confirm = input("[y/n]: ")
            if not(confirm.lower() == "y"):
                continue
            else:
                print("proceeding with operating motor...")
        thought_actions, answer, messages = get_action_steps(task, funcs=dummy_funcs)
        for thought_action in thought_actions:
            thought = thought_action.get("Thought")
            step = thought_action.get("Action")
            print(f"\nThought: {thought}")
            print(f"Action: {step}\n")
            if not (error := step.get("error")):
                obs = invoke_tool(step, real_funcs)
                # obs = invoke_tool(step, dummy_funcs)
            else:
                print(f"Error in executing step: {error}")
        
        if answer:
            print(f"Final answer: {answer}")
            audio = speak(answer)


if False:
    
    dummy_motor, dummy_funcs = get_motor_funcs(dummy=True)
    real_motor, real_funcs = get_motor_funcs(dummy=False)
    
    print(f"Motor connected: {real_motor.connected}")
    
    
    control(real_funcs, dummy_funcs)
    print("Turning off the motor...")
    real_motor.off()

Press [return] to speak, or 'q' to exit:  


Start recording...
End recording...
Time for transcription: 2.905647 secs

Task: Turn left then turn right.

{'blocking': True, 'degrees': -90, 'name': 'run_for_degrees', 'speed': 50}
{'blocking': True, 'degrees': 90, 'name': 'run_for_degrees', 'speed': 50}

Thought: I need to turn left first, which means turning -90 degrees, and then turn right which means turning 90 degrees. I'll use run_for_degrees for both turns.
Action: {'name': 'run_for_degrees', 'degrees': -90, 'speed': 50, 'blocking': True}


Thought: Now that the left turn is complete, I need to turn right by 90 degrees.
Action: {'name': 'run_for_degrees', 'degrees': 90, 'speed': 50, 'blocking': True}

Final answer: Turned left by 90 degrees and then turned right by 90 degrees using run_for_degrees tool.


Press [return] to speak, or 'q' to exit:  q


'q' pressed... Exiting
